In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
#Objetos que dan formato al dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

#Mala práctica hacer Import *
import pyspark.sql.functions

#UDF
from pyspark.sql.functions import udf

In [2]:
sc = SparkContext(master="local", appName="udf")

In [3]:
#Se llama
sqlContext = SQLContext(sc)

A:\Instal_python\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
#Se genera una variable que contiene la ruta donde se encuentran los
var = "C:/Users/Caroline/Videos/curso-apache-spark-platzi/files/"

deptaRDD = sc.textFile(var+"deportistaError.csv").map(lambda line : line.split(","))

In [5]:
#Función para validar el encabezado
#Iter: Devuelve valor a valor lo que se está procesando
def deletHeader(indice, interador):
    return iter(list(interador)[1:])

In [6]:
#Eliminar el encabezado
deptaRDD = deptaRDD.mapPartitionsWithIndex(deletHeader) #Deportista Error

In [7]:
deptaRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '', '', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [8]:
#Antes de Tramsformar los datos del RDD, se debe transformar el tipo de dato
#Deportista Error
deptaRDD = deptaRDD.map(lambda l: (
int(l[0]),
l[1],    
int(l[2]),
int(l[3]),
int(l[4]),
int(l[5]),
int(l[6])
))

#Se crear el Schema
#Deportista Error
depta = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre_deportista", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False),
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

In [9]:
#Aplicar el esquema al RDD
#Deportista Error
deptaRDD = sqlContext.createDataFrame(deptaRDD, depta)

In [13]:
#Funcion para UDF
def conversionEnteros(valor):
    return int(valor) if len(valor) > 0 else 0

#Proceso
conversionEnteros_udf = udf(lambda z: conversionEnteros(z), IntegerType())
#Dar de alta
sqlContext.udf.register("conversionEnteros_udf", conversionEnteros_udf)

<function __main__.<lambda>(z)>

In [15]:
#Uso del UDF
deptaRDD.select(conversionEnteros_udf("genero")).show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "A:\Instal_python\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "A:\Instal_python\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 611, in process
  File "A:\Instal_python\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "A:\Instal_python\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "C:\Users\Caroline\AppData\Local\Temp\ipykernel_1760\1710221715.py", line 8, in <lambda>
ValueError: invalid literal for int() with base 10: ''
